# AutoKeras (AutoML) demos

## Install AutoKeras

In [ ]:
#!pip3 install --upgrade tensorflow pydot pydot_ng autokeras

## AutoKeras Image Classifier

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
import autokeras as ak

clf = ak.ImageClassifier(max_trials=1, overwrite=True)
clf.fit(x_train, y_train)

In [ ]:
model = clf.export_model()
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_layer_names=True, show_shapes=True, show_dtype=True)

In [ ]:
loss, accuracy = clf.evaluate(x_test, y_test)

print(f'Prediction loss: {loss:.4f}')
print(f'Prediction accuracy: {accuracy:.4f}')

In [ ]:
predicted = clf.predict(x_test).flatten().astype('uint8')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
for i in range(100):
    ax = fig.add_subplot(10, 10, i + 1)
    ax.set_axis_off()
    plt.imshow(x_test[i], cmap='binary')
    ax.set_title(f'{predicted[i]}')

plt.tight_layout()
plt.show()

## AutoKeras Text Classifier

In [ ]:
!pip3 install --upgrade nltk

In [ ]:
import pandas as pd

df = pd.read_csv('https://github.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/raw/master/spam.csv', encoding='latin-1')
df

In [ ]:
import re
rule = r'(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?'
f = lambda t: re.sub(rule, '', t)

df.v2 = df.v2.str.lower()
df.v2 = df.v2.apply(f)

In [ ]:
import nltk.corpus
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')
f = lambda t: ' '.join([word for word in t.split() if word not in stop])

df.v2 = df.v2.apply(f)

In [ ]:
df

In [ ]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df.v1 = df.v1.map({'ham': 0, 'spam': 1})
labels = ('ham', 'spam')

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
  df.v2.to_numpy(), df.v1.to_numpy(), test_size=0.2, random_state=42)

In [ ]:
import autokeras as ak

clf = ak.TextClassifier(max_trials=2, overwrite=True)
clf.fit(x_train, y_train)

In [ ]:
model = clf.export_model()
model.summary()

In [ ]:
plot_model(model, show_layer_names=True, show_shapes=True, show_dtype=True)

In [ ]:
loss, accuracy = clf.evaluate(x_test, y_test)

print(f'Prediction loss: {loss:.4f}')
print(f'Prediction accuracy: {accuracy:.4f}')

In [ ]:
predicted = clf.predict(x_test).flatten().astype('uint8')

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted, target_names=labels))

In [ ]:
for i in range(10):
  print('Test:', x_test[i])
  print('Predict:', labels[predicted[i]])
  print('Real:', labels[y_test[i]])
  print('')